In [1]:
import pandas as pd
import numpy as np
import random
import gzip
from collections import defaultdict


In [2]:
df = pd.read_csv('train_Interactions.csv.gz',compression ='gzip')
df['read'] = True
train = df.iloc[:190000]
valid = df.iloc[190000:].reset_index(drop=True)
books_a_user_read = train.groupby('userID')['bookID'].apply(lambda g: g.values.tolist()).to_dict()
users_a_book_has = train.groupby('bookID')['userID'].apply(lambda g: g.values.tolist()).to_dict()
books = list(set(df.bookID.tolist()))
negative = []
for index, row in valid.iterrows():
    user = row['userID']   
    while True:
        book2 = random.choice([x for x in books if x not in books_a_user_read[user]])
        if [user,book2] not in negative:
            negative.append([user, book2, np.NaN, False])
        break
negative = pd.DataFrame(negative, columns=['userID', 'bookID', 'rating', 'read'])
valid = pd.concat([valid,negative])
bookCount = train.bookID.value_counts().to_dict()
totalRead = len(train)
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()
return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalRead/2: break

In [3]:
len(return1)

1398

In [4]:
import pandas as pd
import numpy as np
import random
import gzip
from collections import defaultdict

In [9]:
all_data = pd.read_csv('assignment1/train_Interactions.csv.gz')

In [10]:
all_data['read'] = True

In [11]:
training = all_data[:190000]

In [12]:
validating = all_data[190000:].reset_index(drop=True)

In [15]:
#zip all the pairs into tuples
exist_pair = list(zip(all_data.userID, all_data.bookID))

In [16]:
# randomly generate the pairs that does not exist among all training data.
negative_set = []
for i in list(validating['userID']):
    random_data = all_data.sample(1)
    newBookID = random_data.iloc[0]['bookID']
    existed = False
    while not existed:
        if (((not (i, newBookID) in exist_pair)) and (not (i, newBookID) in negative_set)):
            negative_set.append((i, newBookID))
            existed = True
        else:
            random_data = all_data.sample(1)
            newBookID = random_data.iloc[0]['bookID']

In [17]:
#create the negative dataframe
negative_data = pd.DataFrame(negative_set, columns=['userID', 'bookID'])
negative_data['read'] = False

In [18]:
training.bookID.nunique()

7169

In [19]:
bookCount = training.bookID.value_counts().to_dict()
totalRead = len(training)

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()


In [20]:
return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalRead/2: break

In [21]:
totalRead/2

95000.0

In [22]:
len(return1)

1398

In [15]:
mostPopular[5000]

(13, 'b19334342')

In [16]:
'b84468354' in list(return1)

False

In [285]:
#vaidation set
all_validating = pd.concat([validating, negative_data], sort=False)
Y = all_validating.read.reset_index(drop=True)
vilidate_X = all_validating.bookID.reset_index(drop=True)

In [286]:
predicted_Y = vilidate_X.apply(lambda x: True if x in return1 else False)

In [287]:
len(predicted_Y)

20000

In [288]:
# accuracy of Q1
np.count_nonzero(predicted_Y == Y) / len(Y)

0.4934

In [290]:
# Q2
max_threshold = 0
max_accuracy = 0
for n in np.arange(1, 101):
    return1 = set()
    count = 0
    for ic, i in mostPopular:
      count += ic
      return1.add(i)
      if count > (totalRead * (n/100)): break

    predicted_Y = vilidate_X.apply(lambda x: True if x in return1 else False)
    accuracy = np.count_nonzero(predicted_Y == Y) / len(Y)
    if accuracy > max_accuracy:
        max_accuracy = accuracy
        max_threshold = n

In [291]:
#A better threshold will be
print('A better threshold will be ' + str(max_threshold) + 'th percentile.')
#accuracy
print('The accuracy will be ' + str(max_accuracy))

A better threshold will be 3th percentile.
The accuracy will be 0.50075


In [292]:
#Q3
